# DKTC 최종 제출 파일 생성

## 1. 전체 데이터로 모델 재학습 및 예측

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 데이터 로드
train_df = pd.read_csv('../../data/train.csv')
general_df = pd.read_csv('../../data/train_general_conversation.csv')
test_df = pd.read_csv('../../data/test.csv')
submission_df = pd.read_csv('../../data/submission.csv')

# 전체 학습 데이터 합치기
df_full_train = pd.concat([train_df, general_df], ignore_index=True)

# 클래스 인덱스 정보
class_mapping = {
    '협박 대화': 0,
    '갈취 대화': 1,
    '직장 내 괴롭힘 대화': 2,
    '기타 괴롭힘 대화': 3,
    '일반 대화': 4
}

# 클래스 매핑
df_full_train['class'] = df_full_train['class'].map(class_mapping)

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from mecab_ko import Tagger

# Tokenizer 정의
mecab_tagger = Tagger()
def mecab_tokenizer(text):
    parsed = mecab_tagger.parse(text)
    return [line.split('\t')[0] for line in parsed.split('\n') if '\t' in line]

# TF-IDF 벡터화 (1,3-gram)
print("TF-IDF 벡터화 진행 중... (전체 학습 데이터 사용)")
tfidf_vectorizer = TfidfVectorizer(tokenizer=mecab_tokenizer, ngram_range=(1, 3), min_df=3, max_df=0.9)

# 전체 학습 데이터로 TF-IDF 학습
X_full_train_tfidf = tfidf_vectorizer.fit_transform(df_full_train['conversation'])

# 테스트 데이터 변환
X_test_tfidf = tfidf_vectorizer.transform(test_df['text'])

print("벡터화 완료!")

TF-IDF 벡터화 진행 중... (전체 학습 데이터 사용)


/Users/bychoi/miniforge3/envs/my-pytorch/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


벡터화 완료!


In [16]:
from sklearn.linear_model import LogisticRegression

# 로지스틱 회귀 모델 학습
print("최종 모델 학습 중...")
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_full_train_tfidf, df_full_train['class'])
print("모델 학습 완료!")

# 테스트 데이터 예측
print("테스트 데이터 예측 중...")
test_predictions = lr_model.predict(X_test_tfidf)

print("예측 완료!")

최종 모델 학습 중...


모델 학습 완료!
테스트 데이터 예측 중...
예측 완료!


## 2. 제출 파일 생성

In [17]:
# ## 2. 제출 파일 생성 (수정된 형식)

# 새로운 데이터프레임 생성
submission_final = pd.DataFrame()

# test.csv의 'idx' 컬럼을 사용
submission_final['idx'] = test_df['idx']

# 클래스 이름이 아닌, 예측된 숫자 라벨을 그대로 사용
submission_final['class'] = test_predictions

# 결과 확인
display(submission_final.head())

# 새 파일로 저장
submission_final.to_csv('submission.csv', index=False)

print("submission_numeric.csv 파일이 성공적으로 생성되었습니다.")

,idx,class
0,t_000,1
1,t_001,2
2,t_002,2
3,t_003,3
4,t_004,3


submission_numeric.csv 파일이 성공적으로 생성되었습니다.
